In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
# Load the model, scaler, and ohe
model = load_model('churn_model.h5')
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('ohe_geo.pkl', 'rb') as f:
    ohe_geo = pickle.load(f)

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)



In [3]:
# example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender':'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
print(label_encoder_gender.classes_)

['Female' 'Male']


In [5]:
# Convert input data to DataFrame
input_df = pd.DataFrame([input_data])


# Label encode
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])


In [6]:
# Preprocess the input data
# One-hot encode the Geography column
geo_encoded = ohe_geo.transform(input_df[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(['Geography']))


In [7]:
# Concatenate the one-hot encoded columns with the original DataFrame
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
# scale the input data
input_df_scaled = scaler.transform(input_df)

In [9]:
# predict the churn probability
churn_probability = model.predict(input_df_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step


In [10]:
if churn_probability[0][0] > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is unlikely to churn.")


The customer is unlikely to churn.
